# PERHITUNGAN JARAK 100 DATA SUARA BUKA/TUTUP DAN 1 DATA SUARA BARU MENGGUNAKAN DATA TIME WARPING

Pada tugas ini, data suara yang saya gunakan masih sama dengan tugas sebelumnya. keseluruhan suara saya simpan di folder PSD-audio-wav, yaitu data suara yang telah dikonvert ke ekstensi wav dan terdiri dari suara yang mengucapkan buka atau tutup. Di dalam nya terdaoat 3 folder lain, pertama terdapat folder 'Suara1' yang berisi 50 data rekaman suara saya. folder ini terdiri dari 25 suara buka dengan label 'Perekaman baru 1' sampai dengan 'Perekaman label 25' dan terdiri dari 25 suara tutup dengan label 'Perekaman suara 26' sampai dengan 'Perekaman baru 50'. Folder kedua yaitu 'Suara2" yang berisi 50 data suara teman saya. Folder ini terdiri dari 25 suara buka dengan label 'buka1' sampai dengan 'buka25' dan terdiri dari suara tutup dengan label 'tutup1' sampai dengan 'tutup25'. Ketiga ada folder 'suara-baru' yang terdiri dari 1 rekaman suara buka dengan label 'Perekaman baru 51'. 

In [ ]:
import os
import pandas as pd

base_path = "PSD-audio-wav"

dataset_info = []

folder1 = os.path.join(base_path, "Suara1")
files1 = sorted(os.listdir(folder1))
for i, f in enumerate(files1, start=1):
    label = "buka" if i <= 25 else "tutup"
    dataset_info.append([f, "Suara1", label])

folder2 = os.path.join(base_path, "Suara2")
files2 = sorted(os.listdir(folder2))
for f in files2:
    label = "buka" if f.lower().startswith("buka") else "tutup"
    dataset_info.append([f, "Suara2", label])

folder3 = os.path.join(base_path, "suara-baru")
files3 = sorted(os.listdir(folder3))
for f in files3:
    dataset_info.append([f, "suara-baru", "Tidak Diketahui (Testing)"])

df_dataset = pd.DataFrame(dataset_info, columns=["Nama File", "Folder", "Label"])

jumlah_per_folder = df_dataset["Folder"].value_counts()
print("Jumlah rekaman per folder:")
print(jumlah_per_folder)


Jumlah rekaman per folder:
Folder
Suara1        50
Suara2        50
suara-baru     1
Name: count, dtype: int64


## 1. Import Library

In [4]:
pip install fastdtw


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fastdtw: filename=fastdtw-0.3.4-py3-none-any.whl size=3630 sha256=5a04f4bd8581967d78ec71a1b153d90f738a3c2f84ca38b2bd4677798b5d4a30
  Stored in directory: c:\users\vanisa\appdata\local\pip\cache\wheels\2c\9e\6d\6c419b5fe1b720cf2f9c7b20ea119dc201878ea890ba26221f
Successfully built fastdtw
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'fastdtw' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fastdtw'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install python_speech_features
!pip install scipy


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5907 sha256=7bcfdac7b0c354c469e279a386c6213b79314b863db4fa8cabcc2907274d24cc
  Stored in directory: c:\users\vanisa\appdata\local\pip\cache\wheels\6f\9d\a4\f98b63d53ed4f9ea88655dac2933a0d2c5a18b2d3e9447d0b0
Successfully built python_speech_features


  DEPRECATION: Building 'python_speech_features' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'python_speech_features'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
import librosa
import numpy as np
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io.wavfile as wav
import python_speech_features as psf



## 2. Ekstraksi Audio dan MFCC

In [12]:
import scipy.io.wavfile as wav
import python_speech_features as psf
import numpy as np

def extract_audio_features_no_numba(file_path, n_mfcc=13):
    # Baca audio
    sr, audio = wav.read(file_path)
    
    # Jika stereo, ambil salah satu channel
    if len(audio.shape) == 2:
        audio = audio[:, 0]
    
    # Normalisasi audio ke float (-1, 1)
    audio = audio / np.max(np.abs(audio))
    
    # Ekstraksi MFCC
    mfcc = psf.mfcc(audio, samplerate=sr, numcep=n_mfcc)
    
    return audio, mfcc, sr


## 3. Load Data Training

In [13]:
import os
import pandas as pd

folder_train = "PSD-audio-wav/Suara2"
train_files = sorted(os.listdir(folder_train))

dataset = []

for i, f in enumerate(train_files):
    path = os.path.join(folder_train, f)
    audio, mfcc, sr = extract_audio_features_no_numba(path)
    
    label = "buka" if f.lower().startswith("buka") else "tutup"
    
    dataset.append({
        "id": i+1,
        "filename": f,
        "label": label,
        "length_time_series": len(mfcc),
        "mfcc": mfcc
    })

df_metadata = pd.DataFrame(dataset)
df_metadata.head()


,id,filename,label,length_time_series,mfcc
0,1,buka 1.wav,buka,129,"[[-36.04365338911715, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,2,buka10.wav,buka,198,"[[-36.04365338911715, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,3,buka11.wav,buka,234,"[[-36.04365338911715, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,4,buka12.wav,buka,222,"[[-36.04365338911715, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,5,buka13.wav,buka,193,"[[-36.04365338911715, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## 4. Load Data Baru

In [19]:
def load_audio(file_path, target_sr=16000):
    # Load audio mentah dan resample
    audio, sr = librosa.load(file_path, sr=target_sr)
    return audio, sr

test_file = "PSD-audio-wav/Suara-baru/Perekaman baru 51.wav"
print(f"Audio mentah: {test_audio[:10]} ...")  
print(f"Sampling rate: {sr}")
print(f"Panjang audio (samples): {len(test_audio)}")

Audio mentah: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] ...
Sampling rate: 48000
Panjang audio (samples): 112640


## 5. Normalisasi dan MFCC


In [20]:
def extract_mfcc(audio, sr, n_mfcc=13, nfft=512):
    # Normalisasi
    audio_norm = audio / np.max(np.abs(audio))
    
    # Ekstrak MFCC
    mfcc = psf.mfcc(audio_norm, samplerate=sr, numcep=n_mfcc, nfft=nfft)
    
    return mfcc

test_mfcc = extract_mfcc(test_audio, sr)
print(f"Panjang time series (MFCC): {len(test_mfcc)}")

Panjang time series (MFCC): 234


In [21]:
import librosa
import numpy as np
import python_speech_features as psf

# ===============================
# Fungsi untuk load audio mentah
# ===============================
def load_audio(file_path, target_sr=16000):
    """
    Load audio mentah dan resample ke target_sr
    """
    audio, sr = librosa.load(file_path, sr=target_sr)
    return audio, sr

# ===============================
# Fungsi untuk ekstraksi MFCC + normalisasi
# ===============================
def extract_mfcc(audio, sr, n_mfcc=13, nfft=2048):
    """
    Normalisasi audio dan ekstrak MFCC
    nfft diperbesar untuk menghindari warning
    """
    # Normalisasi audio (-1 sampai 1)
    audio_norm = audio / np.max(np.abs(audio))
    
    # Ekstrak MFCC
    mfcc = psf.mfcc(audio_norm, samplerate=sr, numcep=n_mfcc, nfft=nfft)
    
    return mfcc

# ===============================
# Load file audio baru
# ===============================
test_file = "PSD-audio-wav/Suara-baru/Perekaman baru 51.wav"

# 1️⃣ Load audio mentah
test_audio, sr = load_audio(test_file)
print(f"Audio mentah (10 sample pertama): {test_audio[:10]} ...")
print(f"Sampling rate: {sr}")
print(f"Panjang audio (samples): {len(test_audio)}")

# 2️⃣ Normalisasi + ekstraksi MFCC (nfft lebih besar)
test_mfcc = extract_mfcc(test_audio, sr, n_mfcc=13, nfft=2048)
print(f"Panjang time series (MFCC): {len(test_mfcc)}")


AttributeError: module 'numba' has no attribute 'core'